Advanced Prompt Engineering Problem Set - Unit 1.3
=======================================================================

This problem focuses on market sentiment analysis using multiple data sources
and self-consistency checking. You'll create a system that generates robust
sentiment analysis with confidence scoring.

### Key Concepts to Practice
----------
1. Multi-Source Sentiment Analysis
2. Self-Consistency Checking
3. Confidence Scoring
4. Consensus Generation
5. Cross-Validation Techniques

Let's build a robust sentiment analysis system!

## Step 0: Setup and Dependencies
--------------------------------
First, let's ensure we have all required packages installed.

In [2]:
!pip install numpy pandas matplotlib langchain openai python-dotenv typing-extensions pydantic pydantic_settings langchain-community langchain-openai --quiet

## Step 1: Initial Configuration
--------------------------------
Set up our environment and imports.

In [1]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI

## Step 1.5: Configuration Management
--------------------------------
Set up configuration management for OpenAI credentials in Colab environment.

In [2]:
import os
from typing import Optional
from pydantic_settings import BaseSettings
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

class Settings(BaseSettings):
    """Configuration management for API credentials.

    This class manages API credentials for:
    1. OpenAI
    2. Other services as needed

    Attributes:
        openai_api_key: OpenAI API key
        model_name: OpenAI model identifier
        temperature: Model temperature setting
    """

    # Load variables from .env
    load_dotenv()

    openai_api_key: str = os.getenv('OPENAI_API_KEY')
    model_name: str = os.getenv('MODEL_NAME')
    temperature: float = float(os.getenv('TEMPERATURE'))

In [3]:
def setup_environment() -> ChatOpenAI:
    """Initialize environment and create LLM instance.

    This function:
    1. Loads settings
    2. Sets environment variables
    3. Initializes chat model

    Returns:
        ChatOpenAI: Configured language model instance
    """
    # Load settings
    settings = Settings()

    # Set environment variables
    os.environ["OPENAI_API_KEY"] = settings.openai_api_key

    # Initialize ChatOpenAI with settings
    llm = ChatOpenAI(
        model_name=settings.model_name,
        temperature=settings.temperature
    )

    return llm

In [4]:
# Initialize LLM
try:
    llm = setup_environment()
except Exception as e:
    print(f"Error initializing LLM: {e}")
    print("Please ensure API key is properly set")

## Usage Instructions:
1. Run the pip install cell first
2. Update OPENAI_API_KEY at .env file
3. Run remaining cells to initialize environment
4. Use `llm` instance in your code

## Configuration Tips:
1. Keep API keys secure
2. Update settings as needed
3. Add additional services similarly
4. Manage environment consistently

## Problem 3: Market Sentiment Analysis System
--------------------------------
Design and implement a comprehensive market sentiment analysis system
that combines multiple data sources and ensures consistency.

### Requirements:
1. Multi-Source Analysis:
   - News articles and headlines
   - Social media sentiment
   - Technical indicators
   - Market statistics
   - Analyst reports

2. Self-Consistency Checks:
   - Cross-validation of sources
   - Internal consistency metrics
   - Temporal consistency
   - Source reliability scoring

3. Confidence Scoring:
   - Source-specific confidence
   - Analysis reliability metrics
   - Consensus confidence
   - Time-sensitivity factors

### Template Structure:

In [5]:
from typing import Dict, List, Any
from collections import defaultdict
import numpy as np

class MarketSentimentAnalyzer:
    """A system for comprehensive market sentiment analysis."""

    def __init__(self, llm: Any):
        """Initialize sentiment analysis system."""
        self.llm = llm
        self.source_weights = {
            "news_articles": 0.3,
            "social_media_sentiment": 0.25,
            "technical_indicators": 0.2,
            "market_statistics": 0.15,
            "analyst_reports": 0.1,
        }
        self.source_reliability = defaultdict(lambda: 0.8)  # Default reliability score
        self.temporal_decay_factor = 0.95  # Decay factor for older data

    def analyze_sentiment(self, market_data: Dict[str, str]) -> Dict[str, Any]:
        """Analyze market sentiment from multiple sources."""
        analyses = {}
        for source, content in market_data.items():
            if source == "news_articles":
                analyses[source] = self._analyze_news(content)
            elif source == "social_media_sentiment":
                analyses[source] = self._analyze_social_media(content)
            elif source == "technical_indicators":
                analyses[source] = self._analyze_technical_indicators(content)
            elif source == "market_statistics":
                analyses[source] = self._analyze_market_statistics(content)
            elif source == "analyst_reports":
                analyses[source] = self._analyze_analyst_reports(content)
            else:
                raise ValueError(f"Unknown data source: {source}")

        # Assign confidence scores
        for source, analysis in analyses.items():
            analysis["confidence"] = self._calculate_source_confidence(source, analysis)

        return analyses

    def check_consistency(self, analyses: List[Dict[str, Any]]) -> float:
        """Check consistency between different analyses."""
        sentiment_scores = []
        for analysis in analyses.values():
            sentiment_scores.append(analysis["sentiment_score"])

        # Calculate consistency as the inverse of standard deviation
        consistency = 1 / (np.std(sentiment_scores) + 1e-9)  # Avoid division by zero
        return min(consistency, 1.0)  # Cap consistency at 1.0

    def generate_consensus(self, analyses: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Generate weighted consensus with confidence scores."""
        weighted_sentiment = 0
        total_weight = 0
        for source, analysis in analyses.items():
            weight = self.source_weights[source] * analysis["confidence"]
            weighted_sentiment += analysis["sentiment_score"] * weight
            total_weight += weight

        consensus_sentiment = weighted_sentiment / total_weight
        consistency_score = self.check_consistency(analyses)

        return {
            "consensus_sentiment": consensus_sentiment,
            "consistency_score": consistency_score,
            "confidence": total_weight / sum(self.source_weights.values()),
        }

    def _analyze_news(self, content: str) -> Dict[str, Any]:
        """Analyze sentiment from news articles."""
        # Use LLM to extract sentiment
        response = self.llm(f"Analyze the sentiment of the following news articles:\n{content}")
        sentiment_score = self._extract_sentiment_score(response.content)  # Extract text from AIMessage
        return {"sentiment_score": sentiment_score, "source": "news_articles"}

    def _analyze_social_media(self, content: str) -> Dict[str, Any]:
        """Analyze sentiment from social media."""
        # Initialize default values
        positive_mentions = 0.0
        negative_mentions = 0.0

        # Extract positive mentions
        if "positive mentions" in content:
            try:
                positive_mentions = float(content.split("positive mentions")[1].split("%")[0]) / 100
            except (IndexError, ValueError):
                pass  # Use default value if parsing fails

        # Extract negative mentions
        if "negative mentions" in content:
            try:
                negative_mentions = float(content.split("negative mentions")[1].split("%")[0]) / 100
            except (IndexError, ValueError):
                pass  # Use default value if parsing fails

        # Calculate sentiment score
        sentiment_score = positive_mentions - negative_mentions
        return {"sentiment_score": sentiment_score, "source": "social_media_sentiment"}

    def _analyze_technical_indicators(self, content: str) -> Dict[str, Any]:
        """Analyze sentiment from technical indicators."""
        # Extract RSI and VIX
        rsi = 50.0  # Default value
        vix = 20.0  # Default value

        if "RSI:" in content:
            try:
                rsi = float(content.split("RSI:")[1].split("\n")[0])
            except (IndexError, ValueError):
                pass  # Use default value if parsing fails

        if "VIX:" in content:
            try:
                vix = float(content.split("VIX:")[1].split("\n")[0])
            except (IndexError, ValueError):
                pass  # Use default value if parsing fails

        # Calculate sentiment score
        sentiment_score = (rsi - 50) / 50 - (vix - 20) / 20  # Normalize and combine
        return {"sentiment_score": sentiment_score, "source": "technical_indicators"}

    def _analyze_market_statistics(self, content: str) -> Dict[str, Any]:
        """Analyze sentiment from market statistics."""
        # Extract NASDAQ performance
        nasdaq_change = 0.0  # Default value

        if "NASDAQ:" in content:
            try:
                nasdaq_change = float(content.split("NASDAQ:")[1].split("%")[0]) / 100
            except (IndexError, ValueError):
                pass  # Use default value if parsing fails

        # Use NASDAQ as a proxy for tech sentiment
        sentiment_score = nasdaq_change
        return {"sentiment_score": sentiment_score, "source": "market_statistics"}

    def _analyze_analyst_reports(self, content: str) -> Dict[str, Any]:
        """Analyze sentiment from analyst reports."""
        # Use LLM to extract sentiment
        response = self.llm(f"Analyze the sentiment of the following analyst reports:\n{content}")
        sentiment_score = self._extract_sentiment_score(response.content)  # Extract text from AIMessage
        return {"sentiment_score": sentiment_score, "source": "analyst_reports"}

    def _calculate_source_confidence(self, source: str, analysis: Dict[str, Any]) -> float:
        """Calculate confidence score for a source."""
        base_confidence = self.source_reliability[source]
        temporal_decay = self.temporal_decay_factor  # Adjust based on data freshness
        return base_confidence * temporal_decay

    def _extract_sentiment_score(self, text: str) -> float:
        """
        Extract a sentiment score from LLM response.

        This function sends a text to an LLM to analyze its sentiment and returns a score between -1 and 1.

        Parameters:
        - text (str): The text to analyze for sentiment.

        Returns:
        - float: A sentiment score between -1 and 1.

        Raises:
        - ValueError: If the input text is not a string or if the LLM response is not a valid float between -1 and 1.
        """
        if not isinstance(text, str):
            raise ValueError("Input text must be a string.")

        try:
            response = llm(f"Analyze the sentiment of the following text:\n{text},\
                            please provide only a score between -1 and 1\
                            the score should be a float number")

            # Assuming response.content is the score returned by the LLM
            score = float(response.content)

            if not (-1 <= score <= 1):
                raise ValueError("Sentiment score must be between -1 and 1.")

            return score

        except Exception as e:
            raise ValueError(f"Error in extracting sentiment score: {e}")


### Example Test Data:

In [6]:
market_data = {
    "news_articles": """
HEADLINE: Tech Stocks Rally on Strong Earnings
(Reuters) - Technology stocks surged today following better-than-expected
earnings from major players. Apple Inc. and Microsoft Corp. both beat analyst
estimates, driving broader market gains. Cloud computing and AI segments
showed particular strength.

HEADLINE: Fed Signals Potential Rate Cuts
The Federal Reserve indicated openness to rate cuts later this year, citing
moderating inflation pressures. Markets responded positively to the news,
with bond yields declining.

HEADLINE: Startup Funding Shows Signs of Recovery
Venture capital investments increased 15% in Q4, marking the first
quarterly rise since 2022. Software and fintech sectors led the recovery.
""",

    "social_media_sentiment": """
$AAPL trending positive:
- 65% positive mentions
- 28% neutral mentions
- 7% negative mentions
Volume: 50,000 mentions

$MSFT sentiment metrics:
- 72% positive mentions
- 22% neutral mentions
- 6% negative mentions
Volume: 35,000 mentions

#TechStocks trending topics:
1. #EarningsSeason
2. #TechRally
3. #InvestInTech
""",

    "technical_indicators": """
Market Technical Analysis:
- S&P 500 RSI: 62.5
- NASDAQ RSI: 65.8
- VIX: 16.5
- Moving Averages: Most above 200-day
- Volume: +25% vs 30-day average
- Advance/Decline: 2.5:1
""",

    "market_statistics": """
Market Overview:
- S&P 500: +1.2%
- NASDAQ: +1.8%
- DOW: +0.9%
- Small Caps: +1.5%
- Sector Leaders: Tech +2.3%, Communications +1.9%
- Sector Laggards: Utilities -0.4%, Real Estate -0.2%
""",

    "analyst_reports": """
Goldman Sachs: Overweight Tech Sector
- Target price revisions: +10% average
- Sector outlook: Positive
- Key drivers: AI adoption, cloud growth
- Risk factors: Valuations, rate sensitivity

Morgan Stanley: Market Outlook
- Stance: Constructively bullish
- Focus areas: Quality growth stocks
- Concerns: Geopolitical tensions
- 12-month S&P target: 5200
"""
}

In [7]:
# Initialize the analyzer with an LLM instance
analyzer = MarketSentimentAnalyzer(llm)

analyses = analyzer.analyze_sentiment(market_data)

# Check consistency
consistency_score = analyzer.check_consistency(analyses)

# Generate consensus
consensus = analyzer.generate_consensus(analyses)

print("Consensus Sentiment:", consensus["consensus_sentiment"])
print("Consistency Score:", consensus["consistency_score"])
print("Confidence:", consensus["confidence"])

/var/folders/mn/vp70xqd126nc7g0318l132mh0000gp/T/ipykernel_95959/1965734277.py:75: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.llm(f"Analyze the sentiment of the following news articles:\n{content}")


Consensus Sentiment: 0.4277
Consistency Score: 1.0
Confidence: 0.76


### Implementation Requirements:

1. Code Quality:
   - Clear documentation
   - Type hints
   - Error handling
   - Modular design

2. Analysis Quality:
   - Source-specific processing
   - Robust consistency checks
   - Reliable confidence scoring
   - Clear consensus generation

3. Testing Approach:
   - Multiple data scenarios
   - Edge case handling
   - Time sensitivity tests
   - Cross-validation

4. Output Format:
   - Source-specific sentiments
   - Consistency metrics
   - Confidence scores
   - Final consensus

### Evaluation Criteria:

Your solution will be evaluated on:
1. Implementation completeness
2. Analysis robustness
3. Consistency checking
4. Confidence scoring
5. Code quality

### Tips for Success:

1. Process sources independently
2. Implement thorough validation
3. Weight sources appropriately
4. Handle contradictions clearly
5. Document assumptions

### Common Pitfalls to Avoid:

1. Over-relying on single sources
2. Weak consistency checks
3. Poor confidence scoring
4. Missing edge cases
5. Inadequate validation

### Next Steps:

After completing this problem:
1. Add more data sources
2. Enhance consistency checks
3. Improve confidence scoring
4. Add temporal analysis
5. Implement visualizations